In [69]:
import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Project root added:", PROJECT_ROOT)

Project root added: c:\Users\abdul\Desktop\gradproj\Graduation-Project-2


In [70]:
from nlp.shared.preprocessing import clean_text
from nlp.shared.lexicon_loader import load_lexicon
from nlp.shared.lemmatizer import lemmatize_text
from nlp.shared.lexicon_loader import load_lexicon





In [59]:
clean_text("OUT   of   breath!!!")
lemmatize_text("I am always tired and cannot breathe well")

'I be always tired and can not breathe well'

In [80]:
# ---------------------------
# 0️⃣ Setup
# ---------------------------
import sys
from pathlib import Path
import pandas as pd

project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from nlp.shared.lexicon_loader import load_lexicon
from nlp.shared.symptom_matcher import match_symptoms
from nlp.shared.preprocessing import clean_text
from nlp.shared.lemmatizer import lemmatize_text

# ---------------------------
# 1️⃣ Load Lexicon
# ---------------------------
lexicon_path = project_root / "nlp" / "data" / "lexicon.csv"
lexicon = load_lexicon(str(lexicon_path))

# ---------------------------
# 2️⃣ Load Disease CSVs
# ---------------------------
diabetes_df = pd.read_csv(project_root / "nlp" / "data" / "diabetes_symptoms.csv")
anemia_df = pd.read_csv(project_root / "nlp" / "data" / "anemia_symptoms.csv")

# Convert to dict: disease -> set of symptoms
disease_symptoms = {}

for df in [diabetes_df, anemia_df]:
    for _, row in df.iterrows():
        disease = row['disease'].strip()
        symptoms = set([s.strip() for s in row[1:] if pd.notna(s)])
        disease_symptoms[disease] = symptoms

# ---------------------------
# 3️⃣ Function to map patient text to disease
# ---------------------------
def match_disease(text, lexicon, disease_symptoms):
    # 1️⃣ Lexicon matching
    matched_symptoms = match_symptoms(text, lexicon)
    lexicon_flag = len(matched_symptoms) > 0
    
    # 2️⃣ Disease mapping
    results = []
    if lexicon_flag:
        for disease, symptoms in disease_symptoms.items():
            matched_in_disease = set(matched_symptoms) & symptoms
            if matched_in_disease:
                percentage = len(matched_in_disease) / len(symptoms) * 100
                results.append({
                    "disease": disease,
                    "matched_symptoms": list(matched_in_disease),
                    "percentage": round(percentage, 2)
                })
    return results, lexicon_flag

# ---------------------------
# 4️⃣ Test Example
# ---------------------------
examples = [
    "I get worn out after small activities",
    "My body shuts down after walking",
    "I cut myself and the bleeding does not stop",
    "I am constantly thirsty and urinate frequently"
]

for text in examples:
    results, flag = match_disease(text, lexicon, disease_symptoms)
    print("\nTEXT:", text)
    print("Lexicon matched:", flag)
    if results:
        for r in results:
            print(f"→ Disease: {r['disease']} | Symptoms: {r['matched_symptoms']} | % matched: {r['percentage']}")
    else:
        print("→ No symptoms detected via lexicon, fallback needed")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\abdul\\Desktop\\gradproj\\Graduation-Project-2\\nlp\\nlp\\data\\lexicon.csv'

In [82]:
from nlp.shared.lexicon_loader import load_lexicon
from nlp.shared.symptom_matcher import match_symptoms
from nlp.shared.disease_matcher import match_diseases
import pandas as pd
# Load lexicon (unchanged)
lexicon = load_lexicon("c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/lexicon.csv")

# Load disease files
diabetes = pd.read_csv("c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/diabetes_symptoms.csv")
anemia = pd.read_csv("c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/anemia_symptoms.csv")

disease_symptoms = {}

for df in [diabetes, anemia]:
    for _, row in df.iterrows():
        disease = row["disease"].strip()
        symptoms = set(s.strip() for s in row[1:] if pd.notna(s))
        disease_symptoms[disease] = symptoms
text = "I feel tired and get out of breath easily"

# Step 1: lexicon
detected_symptoms = match_symptoms(text, lexicon)

# Step 2: disease matching
results = match_diseases(detected_symptoms, disease_symptoms)

print("Symptoms:", detected_symptoms)
print("Diseases:", results)

Symptoms: ['Fatigue', 'Shortness of breath']
Diseases: [{'disease': 'anemia', 'matched_symptoms': ['Fatigue'], 'percentage': 10.0}]


In [87]:
import os
import sys

project_root = os.path.dirname(os.getcwd())
sys.path.insert(0, project_root)

# lexicon_path = os.path.join(project_root, "nlp", "data", "lexicon.csv")
# diabetes_path = os.path.join(project_root, "nlp", "data", "diabetes_symptoms.csv")
# anemia_path = os.path.join(project_root, "nlp", "data", "anemia_symptoms.csv")




# ============================================================
# 0️⃣ SETUP (so Jupyter sees nlp/)
# ============================================================

import sys
from pathlib import Path
import pandas as pd

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from nlp.shared.lexicon_loader import load_lexicon
from nlp.shared.symptom_matcher import match_symptoms


# ============================================================
# 1️⃣ LOAD LEXICON (UNCHANGED, WORKING)
# ============================================================

lexicon_path = os.path.join(project_root, "data", "lexicon.csv")
lexicon = load_lexicon(str(lexicon_path))


# ============================================================
# 2️⃣ LOAD DISEASE CSVs
# ============================================================

diabetes_df = pd.read_csv(os.path.join(project_root, "data", "diabetes_symptoms.csv"))
anemia_df   = pd.read_csv(os.path.join(project_root, "data", "anemia_symptoms.csv"))


# ============================================================
# 3️⃣ NORMALIZATION (THIS FIXES YOUR BUG)
# ============================================================

def normalize_symptom(s: str) -> str:
    return s.strip().lower()


# Convert CSVs → disease → set(symptoms)
disease_symptoms = {}

for df in [diabetes_df, anemia_df]:
    for _, row in df.iterrows():
        disease = row["disease"].strip().lower()
        symptoms = {
            normalize_symptom(s)
            for s in row[1:]
            if pd.notna(s)
        }
        disease_symptoms[disease] = symptoms


# ============================================================
# 4️⃣ DISEASE MATCHER (LEXICON FIRST)
# ============================================================

def match_disease(patient_text: str):
    """
    Returns:
    - results: list of diseases with symptoms + percentage
    - lexicon_flag: True if lexicon matched anything
    """

    # Step 1: Lexicon symptom matching
    detected_symptoms = match_symptoms(patient_text, lexicon)
    detected_symptoms = [normalize_symptom(s) for s in detected_symptoms]

    lexicon_flag = len(detected_symptoms) > 0

    if not lexicon_flag:
        return [], False

    detected_set = set(detected_symptoms)
    results = []

    # Step 2: Disease scoring
    for disease, disease_set in disease_symptoms.items():
        matched = detected_set & disease_set
        if matched:
            percentage = len(matched) / len(disease_set) * 100
            results.append({
                "disease": disease,
                "matched_symptoms": sorted(matched),
                "percentage": round(percentage, 2)
            })

    # Sort by percentage (descending)
    results.sort(key=lambda x: x["percentage"], reverse=True)

    return results, True


# ============================================================
# 5️⃣ TEST CASES
# ============================================================

tests = [
    "I am always tired and feel out of breath",
    "I cut myself and the bleeding does not stop",
    "I am constantly thirsty and urinate frequently",
    "I feel weak and dizzy"
]

for text in tests:
    results, flag = match_disease(text)

    print("\nTEXT:", text)
    print("Lexicon matched:", flag)

    if results:
        for r in results:
            print(
                f"→ Disease: {r['disease']} | "
                f"Symptoms: {r['matched_symptoms']} | "
                f"% matched: {r['percentage']}"
            )
    else:
        print("→ No disease matched (semantic fallback later)")



TEXT: I am always tired and feel out of breath
Lexicon matched: True
→ Disease: anemia | Symptoms: ['fatigue', 'shortness of breath'] | % matched: 20.0
→ Disease: diabetes | Symptoms: ['fatigue'] | % matched: 8.33

TEXT: I cut myself and the bleeding does not stop
Lexicon matched: False
→ No disease matched (semantic fallback later)

TEXT: I am constantly thirsty and urinate frequently
Lexicon matched: False
→ No disease matched (semantic fallback later)

TEXT: I feel weak and dizzy
Lexicon matched: True
→ Disease: anemia | Symptoms: ['weakness'] | % matched: 10.0


In [93]:
import os
import sys

project_root = os.path.dirname(os.getcwd())
sys.path.insert(0, project_root)

# lexicon_path = os.path.join(project_root, "nlp", "data", "lexicon.csv")
# diabetes_path = os.path.join(project_root, "nlp", "data", "diabetes_symptoms.csv")
# anemia_path = os.path.join(project_root, "nlp", "data", "anemia_symptoms.csv")




# ============================================================
# 0️⃣ SETUP (so Jupyter sees nlp/)
# ============================================================

import sys
from pathlib import Path
import pandas as pd

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

#from nlp.shared.lexicon_loader import load_lexicon
from nlp.shared.symptom_matcher import match_symptoms


from nlp.shared.lexicon_loader import load_lexicon
from shared.disease_matcher import load_disease_symptoms 
from shared.disease_matcher import match_disease


# Load data

lexicon_path = os.path.join(project_root, "data", "lexicon.csv")
lexicon = load_lexicon(str(lexicon_path))


diabetes_df = pd.read_csv(os.path.join(project_root, "data", "diabetes_symptoms.csv"))
anemia_df   = pd.read_csv(os.path.join(project_root, "data", "anemia_symptoms.csv"))


disease_symptoms = load_disease_symptoms(diabetes_df, anemia_df)

tests = [
    "I am always tired and feel out of breath",
    "I cut myself and the bleeding does not stop",
    "I am constantly thirsty and urinate frequently",
    "I feel weak and dizzy"
]

for text in tests:
    results, flag = match_disease(text, lexicon, disease_symptoms)
    print("\nTEXT:", text)
    print("Lexicon matched:", flag)
    print(results if results else "No disease matched")



TEXT: I am always tired and feel out of breath
Lexicon matched: True
[{'disease': 'anemia', 'matched_symptoms': ['fatigue', 'shortness of breath'], 'percentage': 20.0}, {'disease': 'diabetes', 'matched_symptoms': ['fatigue'], 'percentage': 8.33}]

TEXT: I cut myself and the bleeding does not stop
Lexicon matched: False
No disease matched

TEXT: I am constantly thirsty and urinate frequently
Lexicon matched: False
No disease matched

TEXT: I feel weak and dizzy
Lexicon matched: True
[{'disease': 'anemia', 'matched_symptoms': ['weakness'], 'percentage': 10.0}]


In [94]:
# -----------------------------
# Disease Matcher Test in Notebook
# -----------------------------
import os
from pathlib import Path
import pandas as pd
from shared.disease_matcher_full import DiseaseMatcher

# -----------------------------
# Paths
# -----------------------------
project_root = Path.cwd().parent
lexicon_path  = os.path.join(project_root, "data", "lexicon.csv")
diabetes_path = os.path.join(project_root, "data", "diabetes_symptoms.csv")
anemia_path   = os.path.join(project_root, "data", "anemia_symptoms.csv")

# -----------------------------
# Initialize Matcher
# -----------------------------
matcher = DiseaseMatcher(
    lexicon_path=str(lexicon_path),
    diabetes_path=str(diabetes_path),
    anemia_path=str(anemia_path),
    semantic_model_name="all-MiniLM-L6-v2",
    semantic_threshold=0.55
)

# -----------------------------
# Test Examples
# -----------------------------
tests = [
    "I am always tired and feel out of breath",
    "I cut myself and the bleeding does not stop",
    "I am constantly thirsty and urinate frequently",
    "I feel weak and dizzy"
]

for text in tests:
    results, lexicon_flag = matcher.match_disease(text)
    print("\nTEXT:", text)
    print("Lexicon matched:", lexicon_flag)
    if results:
        for r in results:
            print(f"→ Disease: {r['disease']} | Symptoms: {r['matched_symptoms']} | % matched: {r['percentage']}")
    else:
        print("→ No disease matched")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



TEXT: I am always tired and feel out of breath
Lexicon matched: True
→ Disease: anemia | Symptoms: ['fatigue', 'shortness of breath'] | % matched: 20.0
→ Disease: diabetes | Symptoms: ['fatigue'] | % matched: 8.33

TEXT: I cut myself and the bleeding does not stop
Lexicon matched: False
→ Disease: anemia | Symptoms: ['prolonged bleeding'] | % matched: 10.0

TEXT: I am constantly thirsty and urinate frequently
Lexicon matched: False
→ Disease: diabetes | Symptoms: ['frequent urination', 'increased thirst'] | % matched: 16.67

TEXT: I feel weak and dizzy
Lexicon matched: True
→ Disease: anemia | Symptoms: ['weakness'] | % matched: 10.0


In [98]:
from shared.predictor import predict

predict("I am always tired and feel out of breath")


{'text': 'I am always tired and feel out of breath',
 'lexicon_matched': True,
 'results': [{'disease': 'anemia',
   'matched_symptoms': ['fatigue', 'shortness of breath'],
   'percentage': 20.0},
  {'disease': 'diabetes',
   'matched_symptoms': ['fatigue'],
   'percentage': 8.33}]}

In [ ]:
# ============================================================
# 0️⃣ SETUP (so Jupyter sees nlp/)
# ============================================================

import sys
from pathlib import Path
import pandas as pd

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from nlp.shared.lexicon_loader import load_lexicon
from nlp.shared.symptom_matcher import match_symptoms


# ============================================================
# 1️⃣ LOAD LEXICON (UNCHANGED, WORKING)
# ============================================================

lexicon_path = project_root / "nlp" / "data" / "lexicon.csv"
lexicon = load_lexicon(str(lexicon_path))


# ============================================================
# 2️⃣ LOAD DISEASE CSVs
# ============================================================

diabetes_df = pd.read_csv(project_root / "nlp" / "data" / "diabetes_symptoms.csv")
anemia_df   = pd.read_csv(project_root / "nlp" / "data" / "anemia_symptoms.csv")


# ============================================================
# 3️⃣ NORMALIZATION (THIS FIXES YOUR BUG)
# ============================================================

def normalize_symptom(s: str) -> str:
    return s.strip().lower()


# Convert CSVs → disease → set(symptoms)
disease_symptoms = {}

for df in [diabetes_df, anemia_df]:
    for _, row in df.iterrows():
        disease = row["disease"].strip().lower()
        symptoms = {
            normalize_symptom(s)
            for s in row[1:]
            if pd.notna(s)
        }
        disease_symptoms[disease] = symptoms


# ============================================================
# 4️⃣ DISEASE MATCHER (LEXICON FIRST)
# ============================================================

def match_disease(patient_text: str):
    """
    Returns:
    - results: list of diseases with symptoms + percentage
    - lexicon_flag: True if lexicon matched anything
    """

    # Step 1: Lexicon symptom matching
    detected_symptoms = match_symptoms(patient_text, lexicon)
    detected_symptoms = [normalize_symptom(s) for s in detected_symptoms]

    lexicon_flag = len(detected_symptoms) > 0

    if not lexicon_flag:
        return [], False

    detected_set = set(detected_symptoms)
    results = []

    # Step 2: Disease scoring
    for disease, disease_set in disease_symptoms.items():
        matched = detected_set & disease_set
        if matched:
            percentage = len(matched) / len(disease_set) * 100
            results.append({
                "disease": disease,
                "matched_symptoms": sorted(matched),
                "percentage": round(percentage, 2)
            })

    # Sort by percentage (descending)
    results.sort(key=lambda x: x["percentage"], reverse=True)

    return results, True


# ============================================================
# 5️⃣ TEST CASES
# ============================================================

tests = [
    "I am always tired and feel out of breath",
    "I cut myself and the bleeding does not stop",
    "I am constantly thirsty and urinate frequently",
    "I feel weak and dizzy"
]

for text in tests:
    results, flag = match_disease(text)

    print("\nTEXT:", text)
    print("Lexicon matched:", flag)

    if results:
        for r in results:
            print(
                f"→ Disease: {r['disease']} | "
                f"Symptoms: {r['matched_symptoms']} | "
                f"% matched: {r['percentage']}"
            )
    else:
        print("→ No disease matched (semantic fallback later)")


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\abdul\\Desktop\\gradproj\\Graduation-Project-2\\nlp\\nlp\\data\\lexicon.csv'

In [ ]:
import importlib
from nlp.shared import lexicon_loader

importlib.reload(lexicon_loader)  # reload the updated file

lexicon = lexicon_loader.load_lexicon("c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/lexicon.csv")
for canonical, expressions in lexicon.items():
    print(f"{canonical}: {expressions}")


Fatigue: ['tired', 'exhaust', 'drain', 'no energy', 'always sleepy', 'feel weak all the time']
Shortness of breath: ['breathless', 'hard to breathe', 'can not breathe well', 'out of breath', 'breathe difficulty']
Pale skin: ['skin look pale', 'face be pale', 'lose color', 'light than usual', 'pale face']
Weakness: ['feel weak', 'weak body', 'no strength', 'shaky', 'muscle feel weak']
Frequent infections: ['get sick often', 'catch infection easily', 'always get ill', 'frequent illness']
Rapid or irregular heartbeats: ['fast heartbeat', 'heart racing', 'irregular heartbeat', 'heart beat fast', 'palpitation']
Prolonged bleeding: ['bleeding take long to stop', 'blee a lot from small cut', 'slow blood clotting', 'long bleed time']
Bruising: ['bruise easily', 'bruise appear quickly', 'unexplained bruise', 'bruise without injury']
Jaundice: ['yellow skin', 'yellow eye', 'eye look yellow', 'skin turn yellow']
Decreased exercise tolerance: ['get tired quickly when walk', 'can not exercise much'

In [77]:
from nlp.shared.lexicon_loader import load_lexicon
from nlp.shared.symptom_matcher import match_symptoms

# Load lexicon
lexicon = load_lexicon(r"C:\Users\abdul\Desktop\gradproj\Graduation-Project-2\nlp\data\lexicon.csv")

# Example patient text
text = "I am always tired 😴 and can’t breathe well!"

# Match symptoms
detected = match_symptoms(text, lexicon)
print("Detected Symptoms:", detected)


Detected Symptoms: ['Fatigue', 'Shortness of breath']


In [78]:
from nlp.shared.semantic_utils import semantic_expand

canonical_symptoms = [
    "Fatigue",
    "Shortness of breath"
]

test_texts = [
    "Walking for a few minutes makes my body shut down",
    "I struggle to finish simple tasks without resting",
    "Climbing stairs feels like my lungs stop working",
    "After minimal effort, I feel completely worn out",
    "I need frequent pauses just to catch air"
]

for text in test_texts:
    detected = semantic_expand(text, canonical_symptoms)
    print(f"Text: {text}")
    print(f"Detected symptoms: {detected}\n")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Text: Walking for a few minutes makes my body shut down
Detected symptoms: []

Text: I struggle to finish simple tasks without resting
Detected symptoms: []

Text: Climbing stairs feels like my lungs stop working
Detected symptoms: []

Text: After minimal effort, I feel completely worn out
Detected symptoms: []

Text: I need frequent pauses just to catch air
Detected symptoms: []



In [79]:
canonical_symptoms = [
    "Fatigue",
    "Shortness of breath"
]

texts = [
    "I get worn out after small activities",
    "My body shuts down after walking",
    "I struggle to climb stairs without stopping",
    "Even minor effort leaves me exhausted",
    "I need pauses just to get enough air"
]

for text in texts:
    print(text)
    print(semantic_expand(text, canonical_symptoms))
    print()


I get worn out after small activities
[]

My body shuts down after walking
[]

I struggle to climb stairs without stopping
[]

Even minor effort leaves me exhausted
[]

I need pauses just to get enough air
[]



In [72]:
import importlib
import nlp.shared.semantic_utils as semantic_utils

importlib.reload(semantic_utils)

dir(semantic_utils)


['Dict',
 'List',
 'SemanticMatcher',
 'SentenceTransformer',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__warningregistry__',
 'cosine',
 'pd',
 'spacy',
 'util']

In [86]:
from nlp.shared.semantic_utils import semantic_match


In [90]:
from nlp.shared.lexicon_loader import load_lexicon

lexicon = load_lexicon("C:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/lexicon.csv")
test_sentences = [
    "I get worn out after small activities",
    "My body shuts down after walking",
    "I struggle to climb stairs without stopping",
    "Even minor effort leaves me depleted",
    "I need pauses just to get enough air"
]

for s in test_sentences:
    print(s)
    print(semantic_match(s, lexicon))
    print()


I get worn out after small activities
[{'symptom': 'Fatigue', 'score': 0.781}, {'symptom': 'Shortness of breath', 'score': 0.791}, {'symptom': 'Pale skin', 'score': 0.601}, {'symptom': 'Weakness', 'score': 0.749}, {'symptom': 'Frequent infections', 'score': 0.899}, {'symptom': 'Rapid or irregular heartbeats', 'score': 0.607}, {'symptom': 'Prolonged bleeding', 'score': 0.806}, {'symptom': 'Bruising', 'score': 0.796}, {'symptom': 'Jaundice', 'score': 0.638}, {'symptom': 'Decreased exercise tolerance', 'score': 0.855}, {'symptom': 'Weight loss', 'score': 0.717}, {'symptom': 'Restlessness', 'score': 0.792}, {'symptom': 'Lethargy', 'score': 0.754}, {'symptom': 'Irregular sugar level', 'score': 0.73}, {'symptom': 'Blurred and distorted vision', 'score': 0.774}, {'symptom': 'Obesity', 'score': 0.686}, {'symptom': 'Excessive hunger', 'score': 0.798}, {'symptom': 'Increased thirst', 'score': 0.781}, {'symptom': 'Polyuria', 'score': 0.843}, {'symptom': 'Frequent urination', 'score': 0.843}, {'sy

In [16]:
import os
print(os.path.exists("c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/lexicon.csv"))  # Should print True


True


In [15]:
import os
print(os.getcwd())


c:\Users\abdul\Desktop\gradproj\Graduation-Project-2\nlp\notebooks


In [95]:
from nlp.shared.semantic_utils import semantic_match

tests = [
    "I get worn out after small activities",
    "My body shuts down after walking",
    "I need pauses just to get enough air"
]

for t in tests:
    print(t)
    print(semantic_match(t))
    print()


I get worn out after small activities
[{'symptom': 'Prolonged bleeding', 'score': 0.797}, {'symptom': 'Shortness of breath', 'score': 0.776}, {'symptom': 'Fatigue', 'score': 0.744}, {'symptom': 'Weakness', 'score': 0.605}, {'symptom': 'Decreased exercise tolerance', 'score': 0.605}]

My body shuts down after walking
[{'symptom': 'Shortness of breath', 'score': 0.829}, {'symptom': 'Prolonged bleeding', 'score': 0.783}, {'symptom': 'Fatigue', 'score': 0.773}, {'symptom': 'Decreased exercise tolerance', 'score': 0.689}]

I need pauses just to get enough air
[{'symptom': 'Shortness of breath', 'score': 0.905}, {'symptom': 'Prolonged bleeding', 'score': 0.813}, {'symptom': 'Fatigue', 'score': 0.746}, {'symptom': 'Decreased exercise tolerance', 'score': 0.71}, {'symptom': 'Weakness', 'score': 0.61}]



In [102]:
from nlp.shared.semantic_utils import SemanticMatcher

# Initialize matcher with your CSV paths
matcher = SemanticMatcher(
    types_csv_path="c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_types.csv",
    map_csv_path="c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_type_map.csv"
)

tests = [
    "I get worn out after small activities",
    "My body shuts down after walking",
    "I need pauses just to get enough air",
    "I cut myself and the bleeding does not stop",
    "I am constantly thirsty and urinate frequently"
]

for t in tests:
    print(t)
    print(matcher.semantic_match(t))
    print()


I get worn out after small activities
[{'symptom': 'Shortness of breath', 'score': 0.776}, {'symptom': 'Blurred and distorted vision', 'score': 0.601}]

My body shuts down after walking
[{'symptom': 'Shortness of breath', 'score': 0.829}, {'symptom': 'Weight loss', 'score': 0.69}, {'symptom': 'Obesity', 'score': 0.69}]

I need pauses just to get enough air
[{'symptom': 'Shortness of breath', 'score': 0.905}, {'symptom': 'Blurred and distorted vision', 'score': 0.649}]

I cut myself and the bleeding does not stop
[{'symptom': 'Shortness of breath', 'score': 0.829}, {'symptom': 'Fatigue', 'score': 0.679}, {'symptom': 'Weakness', 'score': 0.679}]

I am constantly thirsty and urinate frequently
[{'symptom': 'Shortness of breath', 'score': 0.799}, {'symptom': 'Fatigue', 'score': 0.705}, {'symptom': 'Weakness', 'score': 0.705}]



In [1]:
# test_semantic.py

from nlp.shared.semantic_utils import SemanticMatcher

# Initialize matcher with CSV files
matcher = SemanticMatcher(
    types_csv_path="c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_types.csv",
    map_csv_path="c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_type_map.csv"
)

# Test sentences (out of lexicon)
tests = [
    "I get worn out after small activities",
    "My body shuts down after walking",
    "I need pauses just to get enough air",
    "I cut myself and the bleeding does not stop",
    "I am constantly thirsty and urinate frequently"
]

for t in tests:
    print(t)
    print(matcher.match_symptoms(t))
    print()


ModuleNotFoundError: No module named 'nlp'

In [2]:
import spacy

nlp = spacy.load("en_core_web_md")
print(nlp("hello").vector.shape)


(300,)


In [36]:
import importlib
from nlp.shared import semantic_utils

importlib.reload(semantic_utils)
from nlp.shared.semantic_utils import SemanticMatcher


In [22]:
matcher = SemanticMatcher(
    "c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_types.csv",
    "c:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_type_map.csv"
)

tests = [
    "I get worn out after small activities",
    "My body shuts down after walking",
    "I need pauses just to get enough air",
    "I cut myself and the bleeding does not stop",
    "I am constantly thirsty and urinate frequently"
]

for t in tests:
    print(t)
    print(matcher.match(t))
    print()


I get worn out after small activities
[]

My body shuts down after walking
[{'symptom': 'Shortness of breath', 'type': 'RESPIRATORY', 'score': 0.627}]

I need pauses just to get enough air


C:\Users\abdul\Desktop\gradproj\Graduation-Project-2\nlp\shared\semantic_utils.py:66: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score = doc.similarity(symptom_doc)


[]

I cut myself and the bleeding does not stop
[{'symptom': 'Slow healing of wounds', 'type': 'FUNCTIONAL', 'score': 0.71}, {'symptom': 'Shortness of breath', 'type': 'RESPIRATORY', 'score': 0.611}]

I am constantly thirsty and urinate frequently
[{'symptom': 'Slow healing of wounds', 'type': 'FUNCTIONAL', 'score': 0.653}, {'symptom': 'Rapid or irregular heartbeats', 'type': 'FUNCTIONAL', 'score': 0.612}]



In [42]:
import importlib
from nlp.shared import semantic_utils
importlib.reload(semantic_utils)

from nlp.shared.semantic_utils import SemanticMatcher


In [41]:
import sys
sys.path.append("C:/Users/abdul/Desktop/gradproj/Graduation-Project-2")


In [43]:
# test_semantic.py
from semantic_utils import SemanticMatcher

def run_tests():
    matcher = SemanticMatcher(
        "C:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_types.csv",
        "C:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_type_map.csv"
    )

    tests = [
        "I get worn out after small activities",
        "My body shuts down after walking",
        "I need pauses just to get enough air",
        "I cut myself and the bleeding does not stop",
        "I am constantly thirsty and urinate frequently"
    ]

    for t in tests:
        print("\nTEXT:", t)
        results = matcher.match(t)
        if not results:
            print("→ NO MATCH")
        else:
            for r in results:
                print(f"→ {r['symptom']} | {r['type']} | score={r['score']}")

if __name__ == "__main__":
    run_tests()


ModuleNotFoundError: No module named 'semantic_utils'

In [44]:
import sys
from pathlib import Path

# notebook is in: nlp/notebooks/
PROJECT_ROOT = Path.cwd().parents[1]   # Graduation-Project-2
sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added:", PROJECT_ROOT)


Project root added: c:\Users\abdul\Desktop\gradproj\Graduation-Project-2


In [45]:
from nlp.shared.semantic_utils import SemanticMatcher


In [46]:
import pandas as pd

df = pd.read_csv("C:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_type_map.csv")
print(df.columns)


Index(['Symptom', 'Type'], dtype='str')


In [55]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

from nlp.shared.semantic_utils import SemanticMatcher

matcher = SemanticMatcher(
    "C:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_types.csv",
    "C:/Users/abdul/Desktop/gradproj/Graduation-Project-2/nlp/data/symptom_type_map.csv"
)

tests = [
    "I get exhausted after minimal effort",
    "I need to stop to catch my breath",
    "Bleeding does not stop after small cuts",
    "I feel thirsty all the time and pee a lot"
]

for t in tests:
    print("\nTEXT:", t)
    for r in matcher.match(t):
        print(f"→ TYPE: {r['type']} | score={r['score']}")
        print(f"  Symptoms: {r['symptoms']}")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



TEXT: I get exhausted after minimal effort
→ TYPE: RESPIRATORY | score=0.491
  Symptoms: ['Shortness of breath']
→ TYPE: SENSATION | score=0.455
  Symptoms: ['Fatigue', 'Weakness']

TEXT: I need to stop to catch my breath
→ TYPE: RESPIRATORY | score=0.432
  Symptoms: ['Shortness of breath']

TEXT: Bleeding does not stop after small cuts

TEXT: I feel thirsty all the time and pee a lot
→ TYPE: METABOLIC | score=0.61
  Symptoms: ['Irregular sugar level', 'Excessive hunger', 'Increased thirst', 'Polyuria', 'Frequent urination', 'Frequent infections']


In [66]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")  # goes from notebooks → nlp
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)


In [78]:
from shared.semantic_utils import SemanticMatcher


In [75]:
matcher = SemanticMatcher(
    symptom_types_csv="../data/symptom_types.csv",
    symptom_map_csv="../data/symptom_type_map.csv"
)


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [59]:
matcher.match("I feel tired")


[{'symptom': 'Fatigue', 'type': 'SENSATION', 'score': 0.603}]

In [60]:
tests = [
    "I get worn out after small activities",
    "I need to stop to catch my breath",
    "Bleeding does not stop after small cuts",
    "I feel thirsty all the time and pee a lot",
    "I feel weak and dizzy",
]

for t in tests:
    print("\nTEXT:", t)
    results = matcher.match(t)
    if not results:
        print("→ No symptoms detected")
    for r in results:
        print(f"→ {r['symptom']} | {r['type']} | score={r['score']}")



TEXT: I get worn out after small activities
→ Decreased exercise tolerance | FUNCTIONAL | score=0.56

TEXT: I need to stop to catch my breath
→ No symptoms detected

TEXT: Bleeding does not stop after small cuts
→ No symptoms detected

TEXT: I feel thirsty all the time and pee a lot
→ Increased thirst | METABOLIC | score=0.697
→ Frequent urination | METABOLIC | score=0.559

TEXT: I feel weak and dizzy
→ No symptoms detected


In [ ]:
# Make sure you give the correct path to your CSV
matcher = SemanticMatcher("../data/symptom_type_map.csv", threshold=0.55)

tests = [
    "I get worn out after small activities",
    "I need to stop to catch my breath",
    "Bleeding does not stop after small cuts",
    "I feel thirsty all the time and pee a lot",
    "I feel weak and dizzy",
]

for t in tests:
    print("\nTEXT:", t)
    res = matcher.match(t)
    if not res:
        print("→ No symptoms detected")
    for r in res:
        print(f"→ {r['symptom']} | {r['type']} | score={r['score']}")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


ValueError: Invalid file path or buffer object type: <class 'float'>

In [69]:
# Search for it
import sys
if 'SemanticMatcher' in dir():
    print("SemanticMatcher exists in current namespace")
    
# Check if it's imported from somewhere
if hasattr(SemanticMatcher, '__module__'):
    print(f"Module: {SemanticMatcher.__module__}")

SemanticMatcher exists in current namespace
Module: shared.semantic_utils
